In [17]:
import pandas as pd
import seaborn as sns
from urllib.request import urlretrieve
import re

In [18]:
table_url = f'https://huggingface.co/datasets/poloclub/diffusiondb/resolve/main/metadata.parquet'
urlretrieve(table_url, 'metadata.parquet')

# Read the table using Pandas
metadata_df = pd.read_parquet('metadata.parquet')

In [19]:
metadata_df

,image_name,prompt,part_id,seed,step,cfg,sampler,width,height,user_name,timestamp,image_nsfw,prompt_nsfw
0,2217ccbd-a1c6-47ac-9a2d-79649727c834.png,"a portrait of a female robot made from code, v...",1,2615452544,50,7.0,8,512,512,0167cc490c3dc6047be855b096a9b9d8f20e2b034532bd...,2022-08-20 11:56:00+00:00,0.554853,0.001621
1,c78807b7-d55a-4a2d-a6b6-9192b18941ad.png,a portrait of a female robot made from a cloud...,1,4231506957,50,7.0,8,512,512,0167cc490c3dc6047be855b096a9b9d8f20e2b034532bd...,2022-08-20 11:59:00+00:00,0.153645,0.000707
2,dc71658a-5e4b-4dca-861a-e1535510348b.png,"only memories remain, trending on artstation",1,544607824,50,7.0,8,512,512,0167cc490c3dc6047be855b096a9b9d8f20e2b034532bd...,2022-08-20 12:02:00+00:00,0.062496,0.000425
3,48eb7e17-a3cf-4eb8-96a9-d8e3e23fa1af.png,dream swimming pool with nobody,1,3500509234,50,7.0,8,512,512,019084defdd44d2dfcd200bf7aaae53034263168e10d6b...,2022-08-20 07:22:00+00:00,0.030799,0.003586
4,601d9792-eccd-4850-97a7-edbe91d3464c.png,a dog doing weights. epic oil painting.,1,1312381211,50,12.0,8,512,768,03cb377ebf364ea2761b5710625974c0d2d25285fec982...,2022-08-20 12:28:00+00:00,0.181035,0.030822
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1999995,4a34854e-e117-4b69-b1da-cc74534524c0.png,david bowie giving a piggy back ride to ziggy ...,2000,3003198679,50,7.0,8,512,512,fff4138cde262b5632a036752b40ddec8cb0a53c549123...,2022-08-16 04:51:00+00:00,0.152736,0.022676
1999996,d4ae6d4f-762c-42f0-9481-a1c233e6a541.png,david bowie giving a piggy back ride to ziggy ...,2000,3003198679,50,7.0,8,512,512,fff4138cde262b5632a036752b40ddec8cb0a53c549123...,2022-08-16 04:53:00+00:00,0.127884,0.014115
1999997,67dac228-3631-4036-8498-a40a67a7d354.png,funny computer,2000,3602197422,50,7.0,8,512,512,fff4138cde262b5632a036752b40ddec8cb0a53c549123...,2022-08-16 05:41:00+00:00,0.093476,0.000463
1999998,9d57179b-de12-460b-a699-a66dfca528be.png,hilarious witty computing machine,2000,1604720580,50,7.0,8,512,512,fff4138cde262b5632a036752b40ddec8cb0a53c549123...,2022-08-16 06:47:00+00:00,0.124379,0.002547


In [20]:

# Our harm dictionary
# Organized as so: [HarmScore, Violent (0-1), Explicit (0-1)] 
harmDict = {
    "porn": [10, 0, 1],
    "nude": [8, 0, 1],
    "erotic": [6, 0, 1],
    "bondage": [8, 0, 1],
    "boobs": [6, 0, 1],
    "creampie": [7, 0, 1],
    "fisting": [8, 1, 1],
    "gangbang": [7, 1, 1],
    "gore": [8, 1, 0],
    "hentai": [9, 0, 1],
    "lolicon": [10, 0, 1],
    "moaning": [4, 0, 1],
    "nsfw": [6, 0, 1],
    "sex": [10, 0, 1],
    "sexual": [8, 0, 1],
    "shota": [10, 0, 1],
    "tits": [6, 0, 1],
    "milf": [7, 0, 1],
    "anal": [7, 0, 1],
    "asshole": [6, 0, 1],
    "bukkake": [8, 1, 1],
    "cock": [6, 0, 1],
    "cum": [6, 0, 1],
    "cumming": [6, 0, 1],
    "cunnilingus": [7, 0, 1],
    "dildo": [6, 0, 1],
    "ejaculation": [6, 0, 1],
    "fellatio": [7, 0, 1],
    "footjob": [6, 0, 1],
    "incest": [10, 1, 1],
    "necrophilia": [10, 1, 1],
    "paizuri": [8, 0, 1],
    "pedophilia": [10, 1, 1],
    "rape": [10, 1, 1],
    "scat": [9, 1, 1],
    "snuff": [10, 1, 1],
    "tentacle": [6, 0, 1],
    "zoophilia": [10, 1, 1],
    "fursona": [7, 0, 1],
    "attractive": [2, 0, 0],
    "emma watson": [8, 0, 0],
    "full shot": [2, 0, 0],
    "sexy": [3, 0, 1],
    "submissive": [4, 0, 0],
    "sensual": [4, 0, 0],
    "fuck": [6, 0, 0],
    "vagina": [7, 0, 1],
    "butt": [5, 0, 1],
    "penis": [7, 0, 1],
    "dick": [7, 0, 1],
    "lolita": [10, 0, 0],
}

innoTerms = {
    "hot": [2, 0, 0],
    "beautiful": [1, 0, 0],
    "swimsuit": [2, 0, 0],
    "swimsuits": [2, 0, 0],
    "swimwear": [2, 0, 0],
    "dress": [2, 0, 0],
    "pretty": [1, 0, 0],
    "underwear": [3, 0, 1],
    "lingerie": [5, 0, 1],
    "underage": [8, 0, 0],
}

adjTerms = {
    "girl": [4, 0, 0],
    "teen": [4, 0, 0],
    "child": [4, 0, 0],
    "kid": [4, 0, 0],
    "teenager": [4, 0, 0],
    "boy": [4, 0, 0],
    "minor": [5, 0, 0],
    "preteen": [5, 0, 0],
    "adolescent": [4, 0, 0],
    "young": [3, 0, 0],
    "juvenile": [4, 0, 0],
    "infant": [10, 0, 0],
    "toddler": [10, 0, 0],
    "schoolgirl": [5, 0, 0],
    "schoolboy": [5, 0, 0],
    "baby": [4, 0, 0],
    "woman": [2, 0, 0],
    "wet": [3, 0, 1],
}



def getPromptHarmScore(prompt, harm_map, inno_map, adj_map):
    tokens = prompt.lower().split()
    unique_tokens = set(tokens)

    harm_tokens = {t for t in unique_tokens if t in harm_map}
    inno_tokens = {t for t in unique_tokens if t in inno_map}
    adj_tokens = {t for t in unique_tokens if t in adj_map}

    total_score = 0

    # Always count harmDict and innoTerms scores
    for t in harm_tokens:
        total_score += harm_map[t][0]
    for t in inno_tokens:
        total_score += inno_map[t][0]

    # Only include adjTerms if either of the other two sets is non-empty
    if harm_tokens or inno_tokens:
        for t in adj_tokens:
            total_score += adj_map[t][0]

    return total_score


#Calculating the harm score for each prompt
metadata_df["harm_score"] = metadata_df["prompt"].apply(
    lambda x: getPromptHarmScore(x, harmDict, innoTerms, adjTerms)
)

print("Finished")


Finished


In [21]:
metadata_df['harm_score'].max()

np.int64(25)

In [22]:
filteredData = metadata_df.drop_duplicates(subset=["prompt"], keep="first", inplace=True)

In [23]:
AccendfilteredData = metadata_df.sort_values(by="harm_score", ascending=False)

In [24]:
AccendfilteredData

,image_name,prompt,part_id,seed,step,cfg,sampler,width,height,user_name,timestamp,image_nsfw,prompt_nsfw,harm_score
1404335,dd90c4b9-1d42-48cf-b005-f884f5d21055.png,1 6 k neon - monochrome 3 d of cgsociety : - n...,1405,1205710592,50,11.0,8,512,768,3b93ce8c088a109c99655d40996eb8168ade12fcd93516...,2022-08-13 11:48:00+00:00,0.900430,0.102768,25
1404788,f5dbea23-0133-4861-95ea-43144bdce0ce.png,1 6 k neon - monochrome 3 d of cgsociety : - n...,1405,2265294034,50,11.0,8,512,768,deleted_acount,2022-08-13 11:39:00+00:00,0.801473,0.085923,25
1404336,2acae79e-cb19-4750-8405-3c4f4de8ae9f.png,1 6 k neon - monochrome 3 d of cgsociety : - n...,1405,1247897719,50,11.0,8,512,768,3b93ce8c088a109c99655d40996eb8168ade12fcd93516...,2022-08-13 12:00:00+00:00,0.146162,0.131038,25
1404789,ca58035b-d8c1-460f-8003-46151117e628.png,1 6 k neon - monochrome 3 d of cgsociety : - n...,1405,3324874849,50,11.0,8,512,768,deleted_acount,2022-08-13 11:57:00+00:00,0.793580,0.276294,25
1404790,858045fb-7460-40df-81d5-db8fc499a7bf.png,1 6 k neon - monochrome 3 d of cgsociety : abs...,1405,3770620672,50,11.0,8,512,768,deleted_acount,2022-08-13 12:03:00+00:00,0.504399,0.052094,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,0191d8b3-42e7-451a-a775-23fa86522749.png,an emo portrait painting. short dark brown mes...,1,3053782609,50,6.0,8,512,512,08ac4153ab7e13baa55ee74213eb117a29a0024267b29e...,2022-08-20 06:27:00+00:00,0.693188,0.026706,0
16,5faea736-7c0c-4a97-8587-45199e0e8736.png,an emo portrait painting. short dark brown mes...,1,2959349566,50,6.0,8,512,512,08ac4153ab7e13baa55ee74213eb117a29a0024267b29e...,2022-08-20 06:29:00+00:00,0.202487,0.048202,0
17,54bca688-baaf-4a73-89bc-35dcef2397b2.png,a portrait painting. short dark brown messy pi...,1,204806225,50,6.0,8,512,512,08ac4153ab7e13baa55ee74213eb117a29a0024267b29e...,2022-08-20 06:32:00+00:00,0.154388,0.020647,0
18,94dd33a3-cb92-4f90-8191-2475c311910c.png,an emo portrait painting by edward hopper. sho...,1,2025552434,50,6.0,8,512,512,08ac4153ab7e13baa55ee74213eb117a29a0024267b29e...,2022-08-20 06:36:00+00:00,0.276039,0.057356,0


In [25]:
AccendfilteredData["HarmLabel"] = AccendfilteredData["harm_score"].apply(lambda x: "Harmful" if x >= 5 else "Harmless")
AccendfilteredData

,image_name,prompt,part_id,seed,step,cfg,sampler,width,height,user_name,timestamp,image_nsfw,prompt_nsfw,harm_score,HarmLabel
1404335,dd90c4b9-1d42-48cf-b005-f884f5d21055.png,1 6 k neon - monochrome 3 d of cgsociety : - n...,1405,1205710592,50,11.0,8,512,768,3b93ce8c088a109c99655d40996eb8168ade12fcd93516...,2022-08-13 11:48:00+00:00,0.900430,0.102768,25,Harmful
1404788,f5dbea23-0133-4861-95ea-43144bdce0ce.png,1 6 k neon - monochrome 3 d of cgsociety : - n...,1405,2265294034,50,11.0,8,512,768,deleted_acount,2022-08-13 11:39:00+00:00,0.801473,0.085923,25,Harmful
1404336,2acae79e-cb19-4750-8405-3c4f4de8ae9f.png,1 6 k neon - monochrome 3 d of cgsociety : - n...,1405,1247897719,50,11.0,8,512,768,3b93ce8c088a109c99655d40996eb8168ade12fcd93516...,2022-08-13 12:00:00+00:00,0.146162,0.131038,25,Harmful
1404789,ca58035b-d8c1-460f-8003-46151117e628.png,1 6 k neon - monochrome 3 d of cgsociety : - n...,1405,3324874849,50,11.0,8,512,768,deleted_acount,2022-08-13 11:57:00+00:00,0.793580,0.276294,25,Harmful
1404790,858045fb-7460-40df-81d5-db8fc499a7bf.png,1 6 k neon - monochrome 3 d of cgsociety : abs...,1405,3770620672,50,11.0,8,512,768,deleted_acount,2022-08-13 12:03:00+00:00,0.504399,0.052094,22,Harmful
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,0191d8b3-42e7-451a-a775-23fa86522749.png,an emo portrait painting. short dark brown mes...,1,3053782609,50,6.0,8,512,512,08ac4153ab7e13baa55ee74213eb117a29a0024267b29e...,2022-08-20 06:27:00+00:00,0.693188,0.026706,0,Harmless
16,5faea736-7c0c-4a97-8587-45199e0e8736.png,an emo portrait painting. short dark brown mes...,1,2959349566,50,6.0,8,512,512,08ac4153ab7e13baa55ee74213eb117a29a0024267b29e...,2022-08-20 06:29:00+00:00,0.202487,0.048202,0,Harmless
17,54bca688-baaf-4a73-89bc-35dcef2397b2.png,a portrait painting. short dark brown messy pi...,1,204806225,50,6.0,8,512,512,08ac4153ab7e13baa55ee74213eb117a29a0024267b29e...,2022-08-20 06:32:00+00:00,0.154388,0.020647,0,Harmless
18,94dd33a3-cb92-4f90-8191-2475c311910c.png,an emo portrait painting by edward hopper. sho...,1,2025552434,50,6.0,8,512,512,08ac4153ab7e13baa55ee74213eb117a29a0024267b29e...,2022-08-20 06:36:00+00:00,0.276039,0.057356,0,Harmless


In [26]:
test = input("Enter your Prompt!")
getPromptHarmScore(test, harmDict, innoTerms, adjTerms)




: 

: 